# Masoscience (advance LSTM model)

**Author:** Mir Yasin Zeinaliyan

**Email:** yasinprodebian@gmail.com  

**Github:** https://github.com/yasin-pro/masoscience

**Description:** In this project, we implement an advanced LSTM model, which is provided for free, but there are no other models in the demo, but we implemented this part completely, and this part is enough to implement the other processes of the project. Unlike this section, although it is a demo, we have presented a very good model.


### Install the necessary tools

To run the codes of this project, you must install the relevant tools

In [ ]:
!pip install keras-tuner

### Import libraries

In this section, I entered the code of all the libraries that are required to run the following codes

In [72]:
import pandas as pd
import numpy as np
import datetime as dt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed, RepeatVector, BatchNormalization, LeakyReLU, Attention, Add
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

### Read data

In this section, we read the prepared data and check it

My data is in my Google Drive, if your data is in another path, you need to change the data reading code

In [73]:
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/My Drive/processed_eurusd.csv")

# df = pd.read_csv("processed_eurusd.csv")

df.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,open,high,low,close,rsi_14,rsi_16,rsi_18,rsi_20,upper_band_18,sma_18,...,adx_18,adx_20,adx_22,adx_24,adx_26,vix_14,vix_16,vix_18,vix_20,change_percent
0,1.4935,1.4946,1.4932,1.4933,51.798561,52.666667,50.803859,52.406417,1.496190,1.491356,...,76.150720,77.154089,81.086150,83.972447,84.639207,2.730207,2.589617,2.454485,2.568109,-0.040179
1,1.4934,1.4940,1.4917,1.4927,50.530035,49.315068,50.000000,55.211268,1.496190,1.491356,...,75.555045,77.404311,80.080729,82.504652,83.396091,2.735378,2.567675,2.458959,2.502187,-0.060293
2,1.4928,1.4930,1.4908,1.4918,45.421245,49.146758,50.158730,48.466258,1.496197,1.491361,...,75.450985,76.530488,78.914289,80.917155,81.891946,2.693769,2.570020,2.456528,2.342697,0.020110
3,1.4919,1.4927,1.4916,1.4921,47.211896,49.491525,48.355263,49.085366,1.496080,1.491306,...,74.871112,75.640679,78.664496,79.777116,80.499218,2.687778,2.571121,2.432486,2.343663,0.000000
4,1.4923,1.4929,1.4919,1.4921,50.396825,46.014493,49.662162,50.628931,1.496060,1.491294,...,74.473970,75.648476,77.847656,78.801339,79.241448,2.643993,2.521053,2.424206,2.331560,-0.147443
5,1.4922,1.4925,1.4888,1.4899,46.350365,43.642612,46.056782,45.092025,1.495930,1.491156,...,73.308506,74.186156,76.186502,77.806649,77.756896,2.717092,2.581479,2.486152,2.366350,0.026847
6,1.4897,1.4915,1.4891,1.4903,56.956522,47.122302,43.377483,46.894410,1.495454,1.490933,...,71.506092,72.863105,75.145562,76.353418,76.382311,2.350803,2.543838,2.442214,2.360614,-0.134201
7,1.4902,1.4903,1.4865,1.4883,48.260870,43.959732,41.587302,43.988270,1.495114,1.490639,...,67.902580,71.248437,73.261449,74.453980,75.127158,2.350692,2.599067,2.487198,2.408145,0.000000
8,1.4884,1.4905,1.4876,1.4883,39.285714,52.400000,43.959732,40.683230,1.494996,1.490439,...,65.534456,69.268066,71.616786,73.123748,73.578248,2.142313,2.262638,2.450424,2.365417,0.094067
9,1.4882,1.4898,1.4877,1.4897,58.333333,51.229508,46.474359,44.072948,1.494826,1.490317,...,63.415609,66.307471,70.329951,71.621675,72.084574,1.543946,2.230307,2.475549,2.383050,-0.073840


### Checking and reviewing data

In this section, we get an overview of the data and check that we have not forgotten anything in the preparation and that the data is ready for learning and performing operations.

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99892 entries, 0 to 99891
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   open            99892 non-null  float64
 1   high            99892 non-null  float64
 2   low             99892 non-null  float64
 3   close           99892 non-null  float64
 4   rsi_14          99892 non-null  float64
 5   rsi_16          99892 non-null  float64
 6   rsi_18          99892 non-null  float64
 7   rsi_20          99892 non-null  float64
 8   upper_band_18   99892 non-null  float64
 9   sma_18          99892 non-null  float64
 10  lower_band_18   99892 non-null  float64
 11  upper_band_20   99892 non-null  float64
 12  sma_20          99892 non-null  float64
 13  lower_band_20   99892 non-null  float64
 14  upper_band_22   99892 non-null  float64
 15  sma_22          99892 non-null  float64
 16  lower_band_22   99892 non-null  float64
 17  upper_band_24   99892 non-null 

### Normalize the data

`RobustScaler` is one of the scalers available in the scikit-learn library, used for scaling features of the data. It is particularly robust in the presence of outliers.

#### How It Works

Unlike `StandardScaler`, which uses the mean and standard deviation, `RobustScaler` uses the median and interquartile range (IQR) for scaling, reducing the influence of outliers on the data.

The formula used by `RobustScaler` to scale each feature is as follows:

---
$$
\hat{x}_i = \frac{x_i - \text{Median}(X)}{\text{IQR}(X)}
$$
---


In [75]:
X = df.drop(["open", "high", "low", "close", "change_percent"], axis=1)
y = df[["open", "high", "low", "close", "change_percent"]]

scaler_X = RobustScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = RobustScaler()
y_scaled = scaler_y.fit_transform(y)

### Data preprocessing

In this section, we divided the data into training and testing sections and transformed it to coordinate with the LSTM model.

In [76]:
X_train, X_test, y_train_base, y_test_base = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

### Model prediction settings

To predict the number of future steps, which refers to predicting the number of future candles, we set the number of outputs and check and adjust the number of outputs for coordination and assurance.

In [77]:
prediction_steps = 5

y_train = np.array([y_train_base[i:i + prediction_steps] for i in range(0, len(y_train_base) - prediction_steps)])
y_test = np.array([y_test_base[i:i + prediction_steps] for i in range(0, len(y_test_base) - prediction_steps)])

X_train = X_train[:len(y_train)]
X_test = X_test[:len(y_test)]

### LSTM model creation

Long Short-Term Memory (LSTM) networks are a type of Recurrent Neural Network (RNN) capable of learning long-term dependencies. They were introduced to mitigate the vanishing gradient problem in traditional RNNs, making them more effective for time series prediction, natural language processing, and other sequential tasks.

This code section builds and trains an advanced LSTM model for time series forecasting, using multiple techniques such as normalization, elimination, regularization, and compression to avoid overfitting and improve model performance.

In [78]:
def attention_3d_block(inputs):
    input_dim = int(inputs.shape[2])
    a = Dense(input_dim, activation='softmax')(inputs)
    output_attention_mul = tf.keras.layers.multiply([inputs, a])
    return output_attention_mul

class LSTMHyperModel(HyperModel):

    def build(self, hp):
        inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))

        x = LSTM(units=hp.Int('units_1', min_value=32, max_value=128, step=32),
                 return_sequences=True, kernel_regularizer=tf.keras.regularizers.L2(0.01))(inputs)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1))(x)

        x = LSTM(units=hp.Int('units_2', min_value=32, max_value=128, step=32),
                 return_sequences=True, kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1))(x)

        x = attention_3d_block(x)

        x = LSTM(units=hp.Int('units_3', min_value=32, max_value=128, step=32),
                 return_sequences=False, kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(hp.Float('dropout_3', min_value=0.2, max_value=0.5, step=0.1))(x)

        x = RepeatVector(prediction_steps)(x)

        x = LSTM(units=hp.Int('units_4', min_value=32, max_value=128, step=32),
                 return_sequences=True, kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(hp.Float('dropout_4', min_value=0.2, max_value=0.5, step=0.1))(x)

        x = LSTM(units=hp.Int('units_5', min_value=32, max_value=128, step=32),
                 return_sequences=True, kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(hp.Float('dropout_5', min_value=0.2, max_value=0.5, step=0.1))(x)

        x = TimeDistributed(Dense(5, kernel_regularizer=tf.keras.regularizers.L2(0.01)))(x)

        model = Model(inputs=inputs, outputs=x)
        model.compile(optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='mean_squared_error')

        return model

### models Callbacks

A few callbacks have been added to make it a good model, especially for storage and significant improvements and...

In [79]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
checkpoint = ModelCheckpoint('lstm_model_checkpoint.keras', monitor='val_loss', save_best_only=True, verbose=1)

### Train model

In this part, the most important part of the program is for the model to learn to use it for prediction

The implemented model is of HyperModel type, so we can find its best settings with existing techniques

In [ ]:
tuner = RandomSearch(
    LSTMHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='lstm_hyperparameter_tuning',
    project_name='lstm_stock_prediction')

tuner.search_space_summary()

tuner.search(X_train, y_train,
             epochs=50,
             validation_split=0.2,
             callbacks=[early_stopping, reduce_lr, checkpoint],
             verbose=1)

Search space summary
Default search space size: 11
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout_1 (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout_2 (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout_3 (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout_4 (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.5, 'step': 0.1,

### Get best model

We get the best model with the following line of code

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

### Save model

We save the seen training model at the end so that we don't run the operation on it every time to use it for prediction. As it turns out, this process is very time-consuming.

I saved it in my Google Drive. To save it in another address, you need to change the following code


In [ ]:
# best_model.save("models/lstm_model.h5")
best_model.save("/content/drive/My Drive/models/lstm_model.h5")